<!-- https://www.documentdb.com/sql/demo -->

# Table of Contents

1. [What is Cosmos DB?](#What-is-Cosmos-DB?)
    1. [More details](#More-details)
2. [Playing with the Python SDK](#Playing-with-the-Python-SDK)
    1. [Setup](#Setup)
3. [Querying with SQL](#Querying-with-SQL)
    1. [SQL syntax](#SQL-syntax)
    2. [Filtering](#Filtering)
    3. [`ORDER BY`](#ORDER-BY)
    4. [`TOP`](#TOP)
    5. [`IN` & `BETWEEN`](#IN-&-BETWEEN)
    6. [Projection](#Projection)
    7. [`JOIN`](#JOIN)
    8. [Built-ins](#Built-ins)
    9. [Geospatial support](#Geospatial-support)

# What is Cosmos DB?
[Cosmos DB](https://docs.microsoft.com/azure/cosmos-db/) (née DocumentDB) is a fully managed NoSQL database service built for fast and predictable performance, high availability, elastic scaling, global distribution, and ease of development. As a schema-free NoSQL database, Cosmos DB provides rich and familiar SQL query capabilities with consistent low latencies on JSON data - ensuring that 99% of your reads are served under 10 milliseconds and 99% of your writes are served under 15 milliseconds. These unique benefits make Cosmoa DB a great fit for web, mobile, gaming, and IoT, and many other applications that need seamless scale and global replication.

Azure Cosmos DB offers the following key capabilities and benefits:
- Elastically scalable throughput and storage (your data is stored on SSDs for low latencies with automatic scaling of your database as needed)
- [Multi-region replication](https://docs.microsoft.com/azure/documentdb/documentdb-distribute-data-globally)
- [Ad-hoc queries with familiar SQL syntax](https://docs.microsoft.com/azure/documentdb/documentdb-sql-query)
- [JavaScript execution within the database](https://docs.microsoft.com/azure/documentdb/documentdb-programming)
- [Tunable consistency levels](https://docs.microsoft.com/azure/documentdb/documentdb-consistency-levels)
- Fully managed (no need to manage any resources related to your database)
- Open by design (all functionality is accessible through JavaScript, JSON, and a REST interface or various SDKs)
- [Automatic indexing](https://docs.microsoft.com/azure/documentdb/documentdb-indexing-policies)

## More details
If you would like to see more details, please read the [introduction to Cosmos DB](https://docs.microsoft.com/azure/cosmos-db/introduction) as well as the [explanation of the resource model and concepts](https://docs.microsoft.com/azure/documentdb/documentdb-resources). You can then follow the [Cosmos DB learning path](https://azure.microsoft.com/documentation/learning-paths/documentdb/) to learn about specific aspects of Cosmos DB from how it works to diving into development.

<!--
 There's also an [overview video from Microsoft Virtual Academy](https://mva.microsoft.com/en-US/training-courses/azure-documentdb-planetscale-nosql-16847) covering Cosmos DB when it was named DocumentDB (embedded below; **note** that if all you see is raw HTML then view the live notebook and run the code cell below).

import IPython.display

IPython.display.IFrame("https://mva.microsoft.com/en-US/training-courses-embed/azure-documentdb-planetscale-nosql-16847/Video-DocumentDB-Overview-mU5LAcT6C_1706218965", 814, 664)
-->

# Playing with the Python SDK
Once you understand what Cosmos DB offers and how it structures data you can then begin to play with a live database. [Creating a Cosmos DB database account](https://docs.microsoft.com/azure/cosmos-db/tutorial-develop-documentdb-dotnet#create-an-azure-cosmos-db-account) is very simple through the [Azure Portal](https://portal.azure.com). Even if you don't have an Azure account, you can [try out a free trial](https://azure.microsoft.com/free/) if you prefer.

But to make trying out Cosmos DB's Python SDK easier, we have a read-only database and a set of examples exploring the various features of Cosmos DB.

## Setup
First, we must install the [`pydocumentdb` package](https://pypi.org/project/pydocumentdb/) (we are working against version 2.0).

In [1]:
!pip --disable-pip-version-check --quiet install --upgrade "pydocumentdb~=2.0.1" "folium~=0.2.1"

With the necessary packages installed, we can now set up a `DocumentClient` which interfaces with our [Cosmos DB account](https://docs.microsoft.com/azure/documentdb/documentdb-resources#database-accounts). At the account level you control the consistency policy and authorization keys (in our instance we are using read-only keys).

In [2]:
from pydocumentdb import document_client

HOST = "https://analytics.documents.azure.com:443/"
READ_ONLY_KEY = "vWP70fGUumeHUoZaXJhoSnsgmjHfZKEXvOIUAS0C3/NKXvuPRn7t/9zT309UVG2Tv7G2wITSnRUwe0UXTqymUw=="

client = document_client.DocumentClient(HOST, {'masterKey': READ_ONLY_KEY})

With a client connection established, we can begin to look at what data is provided. First is the [databases](https://docs.microsoft.com/azure/documentdb/documentdb-resources#databases) themselves. Databases contain users and collections (which will talk about shortly).

In [3]:
list(db["id"] for db in client.ReadDatabases())

INFO:Starting new HTTPS connection (1): analytics.documents.azure.com
INFO:Starting new HTTPS connection (1): analytics-eastus.documents.azure.com


['appservers_metrics',
 'usdanutritiondb',
 'volcanodb',
 'server_metrics',
 'earthquakes']

All of our examples in this notebook will use data stored in the "DG Samples" database. Within each database are [collections](https://docs.microsoft.com/azure/documentdb/documentdb-resources#collections) which act as the transaction domain within a database. This means things such as [store procedures, triggers, and user-defined functions(UDF)](https://docs.microsoft.com/azure/documentdb/documentdb-resources#stored-procedures-triggers-and-user-defined-functions-udf) are defined at the collection level.

In [4]:
DB_ID = "usdanutritiondb"
db = next(db for db in client.ReadDatabases() if db["id"] == DB_ID)

list(coll["id"] for coll in client.ReadCollections(db["_self"]))

['usdanutrition']

Finally, a collection contains [documents](https://docs.microsoft.com/azure/documentdb/documentdb-resources#documents). As Cosmos DB is a NoSQL database, documents do not need to have any specified format whatsoever; in the case of DocumentDB they just need to be [well-formed JSON](http://www.json.org/).

For example, taking a random document from our "food" collection will show you how the document is just JSON data (any key prefixed with `_` is internally used by Cosmos DB).

In [5]:
COLLECTION_ID = "usdanutrition"
food_collection = next(coll for coll in client.ReadCollections(db["_self"]) if coll["id"] == COLLECTION_ID)

next(iter(client.ReadDocuments(food_collection["_self"])))

{'_attachments': 'attachments/',
 '_etag': '"00000800-0000-0000-0000-5620914c0000"',
 '_rid': 'OyAUANaYpAIBAAAAAAAAAA==',
 '_self': 'dbs/OyAUAA==/colls/OyAUANaYpAI=/docs/OyAUANaYpAIBAAAAAAAAAA==/',
 '_ts': 1444974924,
 'description': 'Babyfood, dessert, peach cobbler, junior',
 'foodGroup': 'Baby Foods',
 'id': '03228',
 'isFromSurvey': False,
 'nutrients': [{'description': 'Fiber, total dietary',
   'id': '291',
   'nutritionValue': 0.7,
   'units': 'g'},
  {'description': 'Calcium, Ca',
   'id': '301',
   'nutritionValue': 4,
   'units': 'mg'},
  {'description': 'Copper, Cu',
   'id': '312',
   'nutritionValue': 0.196,
   'units': 'mg'},
  {'description': 'Vitamin D (D2 + D3)',
   'id': '328',
   'nutritionValue': 0,
   'units': 'µg'},
  {'description': 'Vitamin B-6',
   'id': '415',
   'nutritionValue': 0.007,
   'units': 'mg'},
  {'description': 'Folic acid',
   'id': '431',
   'nutritionValue': 1,
   'units': 'µg'},
  {'description': 'Vitamin B-12, added',
   'id': '578',
   'nutr

# Querying with SQL
In the following sections we will cover how to query your Cosmos DB documents using SQL. Since this is a [Jupyter](http://jupyter.org/) notebook you can edit and re-run the code (just make sure you run all cells once to get the variables that are shared across cells defined).

## SQL syntax

To query a collection in Cosmos DB it's easiest to use its SQL syntax. Let’s familiarize ourselves with the classic `SELECT`, `FROM`, and `WHERE` query. The `FROM` is optional and usually set to the current collection’s name (e.g. "food").

In [6]:
def query_collection(collection, client=client):
    def exec_query(query):
        return iter(client.QueryDocuments(collection["_self"], query))
    return exec_query

query_food = query_collection(food_collection)

In [7]:
from pprint import pprint

query = """
SELECT food.id,
    food.description,
    food.tags,
    food.foodGroup
FROM food
WHERE food.foodGroup = "Snacks" and food.id = "19420"
"""

for item in query_food(query):
    pprint(item)

{'description': 'Snacks, granola bars, hard, peanut butter',
 'foodGroup': 'Snacks',
 'id': '19420',
 'tags': [{'name': 'snacks'},
          {'name': 'granola bars'},
          {'name': 'hard'},
          {'name': 'peanut butter'}]}


If you care to experiment, update the query above to fetch the *id*, *description*, *foodGroup*, and *servings* of the document with the *id* of 10188 and *foodGroup* of "Pork Products".

## Filtering
`WHERE` clauses support complex scalar expressions including arithmetic, comparison and logical operators.

In [8]:
query = """
SELECT food.id,
    food.description,
    food.tags,
    food.foodGroup,
    food.manufacturerName,
    food.version
FROM food
WHERE (food.manufacturerName = "The Coca-Cola Company" AND food.version > 0)
"""
for item in query_food(query):
    pprint(item)

{'description': 'Beverages, Energy Drink, sugar-free with guarana',
 'foodGroup': 'Beverages',
 'id': '14026',
 'manufacturerName': 'The Coca-Cola Company',
 'tags': [{'name': 'beverages'},
          {'name': 'energy drink'},
          {'name': 'sugar-free with guarana'}],
 'version': 1}
{'description': 'Energy drink, VAULT, citrus flavor',
 'foodGroup': 'Beverages',
 'id': '14640',
 'manufacturerName': 'The Coca-Cola Company',
 'tags': [{'name': 'energy drink'},
          {'name': 'vault'},
          {'name': 'citrus flavor'}],
 'version': 1}
{'description': 'Beverages, COCA-COLA, POWERADE, lemon-lime flavored, '
                'ready-to-drink',
 'foodGroup': 'Beverages',
 'id': '14461',
 'manufacturerName': 'The Coca-Cola Company',
 'tags': [{'name': 'beverages'},
          {'name': 'coca-cola'},
          {'name': 'powerade'},
          {'name': 'lemon-lime flavored'},
          {'name': 'ready-to-drink'}],
 'version': 1}
{'description': 'Energy drink, VAULT Zero, sugar-free, citru

You can also try querying for food from "Pizza Hut, Inc." and that have a *version* less than 2. (you should have a document with an id of 21271 in the results)

## `ORDER BY`
Cosmos DB supports `Order By` and string range queries. You can sort query results on any string, number, or boolean property.

In [9]:
query = """
SELECT food.description, 
    food.foodGroup, 
    food.servings[0].description AS servingDescription, 
    food.servings[0].weightInGrams AS servingWeight 
FROM food 
WHERE food.foodGroup = "Fruits and Fruit Juices"
    AND food.servings[0].description = "cup"
ORDER BY food.servings[0].weightInGrams DESC
"""

for item in query_food(query):
    pprint(item)

{'description': 'Blueberries, wild, canned, heavy syrup, drained',
 'foodGroup': 'Fruits and Fruit Juices',
 'servingDescription': 'cup',
 'servingWeight': 319}
{'description': 'Prunes, dehydrated (low-moisture), stewed',
 'foodGroup': 'Fruits and Fruit Juices',
 'servingDescription': 'cup',
 'servingWeight': 280}
{'description': 'Apples, dried, sulfured, stewed, with added sugar',
 'foodGroup': 'Fruits and Fruit Juices',
 'servingDescription': 'cup',
 'servingWeight': 280}
{'description': 'Cranberry sauce, canned, sweetened',
 'foodGroup': 'Fruits and Fruit Juices',
 'servingDescription': 'cup',
 'servingWeight': 277}
{'description': 'Cranberry-orange relish, canned',
 'foodGroup': 'Fruits and Fruit Juices',
 'servingDescription': 'cup',
 'servingWeight': 275}
{'description': 'Peaches, canned, heavy syrup pack, solids and liquids',
 'foodGroup': 'Fruits and Fruit Juices',
 'servingDescription': 'cup',
 'servingWeight': 262}
{'description': 'Fruit salad, (peach and pear and apricot and

Try returning the *description*, *foodGroup*, and *tags[0].name* for all "Fruit and Fruit Juices" with *servings[0].description* as "cup" in ascending order based on *tags[0].name*. (you should find a food named "acerola" in your results)

## `TOP`
`TOP` can be used to limit the number of returning values from a query.

In [10]:
query = """
SELECT TOP 5 food.id, 
    food.description, 
    food.tags, 
    food.foodGroup 
FROM food 
WHERE food.foodGroup = "Snacks"
"""

for item in query_food(query):
    pprint(item)

{'description': 'Snacks, popcorn, air-popped',
 'foodGroup': 'Snacks',
 'id': '19034',
 'tags': [{'name': 'snacks'}, {'name': 'popcorn'}, {'name': 'air-popped'}]}
{'description': 'Snacks, popcorn, cakes',
 'foodGroup': 'Snacks',
 'id': '19036',
 'tags': [{'name': 'snacks'}, {'name': 'popcorn'}, {'name': 'cakes'}]}
{'description': 'Snacks, pretzels, hard, plain, salted',
 'foodGroup': 'Snacks',
 'id': '19047',
 'tags': [{'name': 'snacks'},
          {'name': 'pretzels'},
          {'name': 'hard'},
          {'name': 'plain'},
          {'name': 'salted'}]}
{'description': 'Snacks, rice cakes, brown rice, plain',
 'foodGroup': 'Snacks',
 'id': '19051',
 'tags': [{'name': 'snacks'},
          {'name': 'rice cakes'},
          {'name': 'brown rice'},
          {'name': 'plain'}]}
{'description': 'Snacks, potato chips, made from dried potatoes, cheese-flavor',
 'foodGroup': 'Snacks',
 'id': '19412',
 'tags': [{'name': 'snacks'},
          {'name': 'potato chips'},
          {'name': 'made 

Now try returning the *id*, *description*, *tags*, *foodGroup*, and *servings[0]* as "serving" for the top 10 "Snacks" with *servings[0].description* as "cup" in descending order based on *servings[0].weightInGrams*. Use `DESC` at the end of the `ORDER BY` query to get descending order. (you should find a food with the ID of 19821 in your results)

## `IN` & `BETWEEN`
`IN` can be used to check whether a specified value matches any element in a given list and `BETWEEN` can be used to run queries against a range of values.

In [11]:
query = """
SELECT food.id,
    food.description,
    food.tags,
    food.foodGroup,
    food.version 
FROM food 
WHERE food.foodGroup 
    IN ("Poultry Products", "Sausages and Luncheon Meats")
    AND (food.id BETWEEN "05740" AND "07050")
"""

for item in query_food(query):
    pprint(item)

{'description': 'Turkey, thigh, from whole bird, meat only, roasted',
 'foodGroup': 'Poultry Products',
 'id': '05741',
 'tags': [{'name': 'turkey'},
          {'name': 'thigh'},
          {'name': 'from whole bird'},
          {'name': 'meat only'},
          {'name': 'roasted'}],
 'version': 1}
{'description': 'Turkey, retail parts, thigh, meat and skin, cooked, roasted',
 'foodGroup': 'Poultry Products',
 'id': '05743',
 'tags': [{'name': 'turkey'},
          {'name': 'retail parts'},
          {'name': 'thigh'},
          {'name': 'meat and skin'},
          {'name': 'cooked'},
          {'name': 'roasted'}],
 'version': 1}
{'description': 'Chicken, broiler or fryers, breast, skinless, boneless, meat '
                'only, enhanced, cooked, braised',
 'foodGroup': 'Poultry Products',
 'id': '05748',
 'tags': [{'name': 'chicken'},
          {'name': 'broiler or fryers'},
          {'name': 'breast'},
          {'name': 'skinless'},
          {'name': 'boneless'},
          {'name'

Try returning the *id*, *description*, *tags*, *foodGroup*, and *version* for all "Breakfast Cereals" or "Fruits and Fruit Juices" with an id between 08700 and 09050. (you should find an item with an ID of 09002 in your results)

## Projection
Cosmos DB supports projection with JSON within its queries.

In [12]:
query = """
SELECT { 
    "Company": food.manufacturerName,
    "Brand": food.commonName,
    "Serving Description": food.servings[0].description,
    "Serving in Grams": food.servings[0].weightInGrams,
    "Food Group": food.foodGroup 
    } AS Food
FROM food
WHERE food.id = "08640"
"""

for item in query_food(query):
    pprint(item)

{'Food': {'Brand': 'instant oatmeal, not fortified, no added salt',
          'Company': 'The Quaker Oats, Co.',
          'Food Group': 'Breakfast Cereals',
          'Serving Description': 'packet',
          'Serving in Grams': 41}}


Now try a query where you set the new "Brand" property to *food.tags[0].name* and create a new "Menu Item" property for *food.tags[1].name*. Keep using the document where id is 21437 (your result should have the "Menu Item" as "fried chicken")

## `JOIN`
Cosmos DB’s `JOIN` supports intra-document and self-joins.

In [13]:
query = """
SELECT tag.name
FROM food
JOIN tag IN food.tags
WHERE food.id = "08640"
"""

for item in query_food(query):
    pprint(item)

{'name': 'cereals'}
{'name': 'quaker'}
{'name': 'instant oatmeal organic'}
{'name': 'regular'}


Now try returning the *id*, *description*, *nutritionValue*, and *units* for *food.nutrients* with a *nutritionValue* greater than 5 and units in "mg" for the document with an ID of 03228. (you should expect a result have a description of "Phosphorus, P")

## Built-ins
Cosmos DB supports a number of [built-in functions](https://docs.microsoft.com/en-ca/azure/documentdb/documentdb-sql-query#a-idbuiltinfunctionsabuilt-in-functions) for common operations. They cover mathematical functions like `ABS`, `FLOOR`, and `ROUND` and type checking functions like `IS_ARRAY`, `IS_BOOL` and `IS_DEFINED`.

In [14]:
query = """
SELECT food.id, 
    food.commonName, 
    food.foodGroup, 
    ROUND(nutrient.nutritionValue) AS amount, 
    nutrient.units 
FROM food JOIN nutrient IN food.nutrients 
WHERE IS_DEFINED(food.commonName) 
    AND nutrient.description = "Water" 
    AND food.foodGroup IN ("Sausages and Luncheon Meats", "Legumes and Legume Products")
    AND food.id > "42178"
"""

for item in query_food(query):
    pprint(item)

{'amount': 71,
 'commonName': 'hot dog, wiener, frank',
 'foodGroup': 'Sausages and Luncheon Meats',
 'id': '42188',
 'units': 'g'}
{'amount': 70,
 'commonName': 'hot dog, wiener, frank',
 'foodGroup': 'Sausages and Luncheon Meats',
 'id': '42280',
 'units': 'g'}


## Geospatial support

Cosmos DB supports [geospatial indexing and querying](https://azure.microsoft.com/documentation/articles/documentdb-geospatial/). You can store location data in your schema-less JSON documents and perform proximity queries. (**NOTE**: If you don't see any output or just raw HTML, run the live notebook either through `Cell ➞ Run All` or manually run the cells in the *Setup* section and the cells below)

In [15]:
import folium

world = folium.Map(location=[45.5236, -122.6750], tiles='Stamen Terrain', zoom_start=1)

In [16]:
volcano_db = next(db for db in client.ReadDatabases() if db["id"] == "volcanodb")
volcanoes = next(coll for coll in client.ReadCollections(volcano_db["_self"]) if coll["id"] == "volcano5")
query_volcanoes = query_collection(volcanoes)

# Find all volcanoes of type Stratovolcano
# (http://www.sciencedaily.com/terms/stratovolcano.htm) 
# that are within 100 km of Redmond, WA. 
query = """
SELECT *
FROM volcanoes v
WHERE ST_DISTANCE(v.Location, { 
    "type": "Point", 
    "coordinates": [-122.19, 47.36] 
    }) < 100 * 1000
AND v.Type = "Stratovolcano"
AND v["Last Known Eruption"] = "Last known eruption from 1800-1899, inclusive"
"""

found = []
for item in query_volcanoes(query):
    lng, lat = item["Location"]["coordinates"]  # Cosmos DB uses [longitude, latitude], Folium uses the reverse!
    found.append([lat, lng])
    world.add_children(folium.Marker([lat, lng], popup=item["Volcano Name"]))
northeast = [max(coord[0] for coord in found), max(coord[1] for coord in found)]
southwest = [min(coord[0] for coord in found), min(coord[1] for coord in found)]
world.fit_bounds([southwest, northeast], max_zoom=5)
world